In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('float_format', '{:f}'.format)

In [ ]:
def get_cols(df: pd.DataFrame) -> tuple:
    numeric_columns = df.select_dtypes(include=np.number).columns.tolist()
    categorical_columns = df.select_dtypes(exclude=np.number).columns.tolist()

    return numeric_columns, categorical_columns

# *balances*

In [ ]:
df_balances = (
    pd.read_csv('../data/raw/balances.csv')
    [['PERIODO','ID','CANT_EMP_NEG','CANT_EMP_CONS','CANT_EMP_HIPOT','SALDO_MED_EMP','SALDO_PEQ_EMP','SALDO_MIC_EMP','SALDO_CONS_REV','SALDO_CONS_NO_REV','SALDO_HIPOT','SALDO_VENCIDO']]
)

"""
PERIODO - Periodo de extracción de los datos
ID - Identificador único

CANT_EMP_NEG - Cantidad de empresas de negocio
CANT_EMP_CONS - Cantidad de empresas de consumo
CANT_EMP_HIPOT - Cantidad de empresas hipotecario
    * last
    * variable para ver si cerro o abrio empresas en los ultimos 9 meses

SALDO_MED_EMP - Saldo en mediana empresa
SALDO_PEQ_EMP - Saldo en pequeña empresa
SALDO_MIC_EMP - Saldo en micro empresa
    * categorica para saber que tipo de empresa tiene
    * last
    * variacion promedio de los saldos en los saldos que tiene

SALDO_CONS_REV - Saldo Consumo revolvente
SALDO_CONS_NO_REV - Saldo Consumo NO revolvente
    
SALDO_HIPOT - Saldo en hipotecario
SALDO_VENCIDO - Saldo vencido

CANT_EMP_DOL_NEG - Cantidad de empresas de negocios en dólares
SALDO_DOLA_NEG - Saldo en dólares de negocio
CANT_EMP_DOL_CONS - Cantidad de empresas de consumo en dólares
SALDO_DOLA_CONS - Saldo en dólares de consumo
CANT_EMP_DOL_HIPOT - Cantidad de empresas de hipotecario en dólares
SALDO_DOLA_HIPOT - Saldo en dólares de hipotecario
MAX_LINEA_DISP_U6M - Línea Máxima disponible en los últimos 6 meses
"""


In [ ]:
df_balances['SALDO_EMP_TOTAL'] = df_balances['SALDO_MED_EMP'] + df_balances['SALDO_PEQ_EMP'] + df_balances['SALDO_MIC_EMP'] + df_balances['SALDO_HIPOT'] + df_balances['SALDO_CONS_REV'] + df_balances['SALDO_CONS_NO_REV']

df_balances['SALDO_EMP_TOTAL-VENCIDO'] = df_balances['SALDO_EMP_TOTAL'] - df_balances['SALDO_VENCIDO']
df_balances['SALDO_REV_NO_REV-VENCIDO'] = df_balances['SALDO_CONS_REV'] + df_balances['SALDO_CONS_NO_REV'] - df_balances['SALDO_VENCIDO']
df_balances['SALDO_TOTAL-VENCIDO'] = df_balances['SALDO_EMP_TOTAL'] + df_balances['SALDO_CONS_REV'] + df_balances['SALDO_CONS_NO_REV'] - df_balances['SALDO_VENCIDO']

In [ ]:
df_balances['TIENE_SALDO_VENCIDO'] = 0
indexes_SALDO_VENCIDO = df_balances[df_balances['SALDO_VENCIDO'] > 0].index
df_balances.loc[indexes_SALDO_VENCIDO, 'TIENE_SALDO_VENCIDO'] = 1
df_balances['CANT_MESES_SALDO_VENCIDO'] = df_balances.groupby('ID')['TIENE_SALDO_VENCIDO'].transform('sum')
df_balances = df_balances.drop(columns=['TIENE_SALDO_VENCIDO'])

In [ ]:
df_balances.head(9*4)

In [ ]:
# 'SALDO_EMP_TOTAL', 'SALDO_PEQ_EMP', 'SALDO_MED_EMP', 'SALDO_MIC_EMP', 'SALDO_CONS_REV', 'SALDO_CONS_NO_REV', 'SALDO_HIPOT', 'SALDO_VENCIDO'

for col in ['SALDO_EMP_TOTAL', 'SALDO_EMP_TOTAL-VENCIDO', 'SALDO_REV_NO_REV-VENCIDO', 'SALDO_TOTAL-VENCIDO']:
    print(col)
    df_balances[f'VAR_{col}_MA8'] = df_balances.groupby('ID')[f'{col}'].transform(lambda x: x.diff().rolling(window=8).mean())
    
df_balances.head(9*4)

In [ ]:
df_balances.to_parquet('../data/processed/balances_v2.gzip', compression='gzip')

In [ ]:
df_balances.head(50)

In [ ]:
df_balances.describe()

In [ ]:
df_balances.dtypes

In [ ]:
df_balances.isnull().sum() / len(df_balances)

In [ ]:
df_aux = df_balances.copy().replace(0, np.nan)
msno.matrix(df_aux, figsize=(10, 5))

In [ ]:
df_balances = pd.read_parquet('../data/processed/balances.gzip')
df_balances.head(9*3)

In [ ]:
df_balances.columns

In [ ]:
df_balances_unirow = df_balances.groupby('ID').agg(
    {
        'CANT_EMP_NEG': ['max', 'last'],
        'CANT_EMP_CONS': ['max', 'last'],
        'CANT_EMP_HIPOT': ['max', 'last'],
        'CANT_MESES_SALDO_VENCIDO': ['max'],
        'SALDO_MED_EMP': ['mean'],
        'SALDO_PEQ_EMP': ['mean'],
        'SALDO_MIC_EMP': ['mean'],
        'SALDO_CONS_REV': ['mean'],
        'SALDO_CONS_NO_REV': ['mean'],
        'SALDO_HIPOT': ['mean'],
        'SALDO_VENCIDO': ['mean'],

        'SALDO_EMP_TOTAL': ['mean'],
        
        'VAR_SALDO_EMP_TOTAL_MA8' : ['mean'], 
        'VAR_SALDO_EMP_TOTAL-VENCIDO_MA8' : ['mean'],
        'VAR_SALDO_REV_NO_REV-VENCIDO_MA8' : ['mean'],
        'VAR_SALDO_TOTAL-VENCIDO_MA8' : ['mean'],

    }
).replace([np.inf, -np.inf], np.nan).fillna(0)

df_balances_unirow.columns = ['_'.join(col) for col in df_balances_unirow.columns.values]
df_balances_unirow = df_balances_unirow.reset_index()

df_balances_unirow

In [ ]:
nc, cc = get_cols(df_balances_unirow)
df_balances_unirow[nc].std(numeric_only=True).reset_index().sort_values(by=0, ascending=False)

In [ ]:
import numpy as np

corr_mat = df_balances_unirow[nc].corr(method='spearman')
mask = np.tril(np.ones_like(corr_mat, dtype=bool))  # create a mask to hide the lower triangle of the heatmap
plt.figure(figsize=(40, 40))
sns.heatmap(corr_mat, annot=True, cmap='coolwarm', vmin=-1, vmax=1, square=True, fmt='.2f', mask=mask)  # add the mask to the heatmap
plt.title('Imputed Correlation Matrix')
plt.show()

# *customers*

In [ ]:
df_customers = (
    pd.read_csv('../data/raw/customers.csv')
    .assign(
        PER_BANCARIZACION = lambda row: row['PER_BANCARIZACION'].astype(str)
    )
    .assign(
        ANIO_BANCARIZACION = lambda row: row['PER_BANCARIZACION'].str[:4],
        MES_BANCARIZACION = lambda row: row['PER_BANCARIZACION'].str[5:7]
    )
    .assign(
        ANIO_BANCARIZACION = lambda row: row['ANIO_BANCARIZACION'].astype(int),
        MES_BANCARIZACION = lambda row: row['MES_BANCARIZACION'].astype(int)
    )
    .drop(columns=['PER_BANCARIZACION'])
)
df_customers['CO_TIPO_SEXO'] = df_customers['CO_TIPO_SEXO'].astype('category')

df_customers

In [ ]:
df_customers.dtypes

In [ ]:
df_customers['CO_TIPO_SEXO'].value_counts(normalize=True)

In [ ]:
df_customers['NO_DEPARTAMENTO'].value_counts(normalize=True)
# * IDEA: Agrupar por regiones

In [ ]:
display(df_customers['DE_CIIU'].value_counts(normalize=True))

In [ ]:
msno.matrix(df_customers, figsize=(10, 5))

# *JOIN*

In [ ]:
df_join = df_customers.drop(columns=['NO_DEPARTAMENTO', 'NO_PROVINCIA', 'DE_CIIU']).merge(
    df_balances_unirow, how='left', on='ID'
)

In [ ]:
df_join['EDAD'].describe()

In [ ]:
sns.kdeplot(df_join['EDAD'], shade=True)

In [ ]:
# # fillna EDAD with KNN
# from sklearn.impute import KNNImputer

# imputer = KNNImputer(n_neighbors=5)
# df_join['EDAD'] = imputer.fit_transform(df_join[['EDAD']])
# sns.kdeplot(df_join['EDAD'], shade=True)

In [ ]:
df_join.isna().sum()

In [ ]:
nc, cc = get_cols(df_join)
df_join[nc].std(numeric_only=True).reset_index().sort_values(by=0, ascending=False)

In [ ]:
df_join.dtypes

In [ ]:
df_train = pd.read_csv('../data/raw/train.csv').merge(df_join, how='left', on='ID')
df_test = pd.read_csv('../data/raw/test.csv').merge(df_join, how='left', on='ID')

In [ ]:
df_test.isna().sum()

In [ ]:
df_train.isna().sum()

In [ ]:
display(df_train['TARGET'].value_counts(normalize=True))
display(df_test['TARGET'].value_counts(normalize=True))

In [ ]:
df_train.to_parquet('../data/processed/train_v2.gzip', compression='gzip')
df_test.to_parquet('../data/processed/test_v2.gzip', compression='gzip')

In [ ]:
df_test.isna().sum()

In [ ]:
df_train.isna().sum()

In [ ]:
df_train['ID'].isin(df_balances['ID']).value_counts()

In [ ]:
df_test['ID'].isin(df_balances['ID']).value_counts()

In [ ]:
df_train['ID'].isin(df_customers['ID']).value_counts()

In [ ]:
df_test['ID'].isin(df_customers['ID']).value_counts()

In [ ]:
df_train.dtypes